In [1]:
import os 
import pandas as pd
import numpy as np

path = os.getcwd()

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path = 'C:\\Users\\sunil\\Projects\\Dockship\\segmind_grand_ai_challenge_2021-dataset'

In [3]:
train = pd.read_csv(path + "\\dataset\\TRAIN.csv")
test = pd.read_csv(path + "\\dataset\\TEST.csv")
ss = pd.read_csv(path + "\\dataset\\sample_submission.csv")
shop = pd.read_csv(path + "\\dataset\\store.csv")

C:\Users\sunil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# train['Date'] = pd.to_datetime(train['Date'])
# test['Date'] = pd.to_datetime(test['Date'])


# train = pd.merge(train, shop, how = 'left', on = 'Store')
# test = pd.merge(test, shop, how = 'left', on = 'Store')


# df = pd.concat([train, test], axis = 0).reset_index(drop=True)

In [5]:
# def get_elapsed(fld, pre):
#     day1 = np.timedelta64(1, 'D')
#     last_date = np.datetime64()
#     last_store = 0
#     res = []
    
#     for s, v, d in zip(df.Store.values, df[fld].values, df.Date.values): # zip is 300 times faster than using for rows in df.iterrows():
#         if s!= last_store:
#             last_date = np.datetime64()
#             last_store = s
#         if v: last_date = d # keep track of the row if it is a school holiday
#         res.append((d-last_date).astype('timedelta64[D]') / day1) # append the no of days since last school holiday to result(res)
#     df[pre+fld] = res

# columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]
# #df = train[columns]
# df = train[columns].append(test[columns])

# fld = 'SchoolHoliday'
# df = df.sort_values(['Store', 'Date'])
# get_elapsed(fld, 'After')
# df = df.sort_values(['Store', 'Date'], ascending=[True, False]) # sort store in scending and date in descending order.
# get_elapsed(fld, 'Before')

# fld = 'StateHoliday'
# df = df.sort_values(['Store', 'Date'])
# get_elapsed(fld, 'After')
# df = df.sort_values(['Store', 'Date'], ascending=[True, False])
# get_elapsed(fld, 'Before')


# fld = 'Promo'
# df = df.sort_values(['Store', 'Date'])
# get_elapsed(fld, 'After')
# df = df.sort_values(['Store', 'Date'], ascending=[True, False])
# get_elapsed(fld, 'Before')


# df = df.set_index("Date")



# columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

# for o in ['Before', 'After']:
#     for p in columns:
#         a = o+p
#         df[a] = df[a].fillna(0).astype(int)

# bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

# bwd.drop('Store',1,inplace=True)
# bwd.reset_index(inplace=True)

# df = df.merge(bwd, 'left', ["Date", "Store"], suffixes=['', '_bw'])

In [6]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
train['week'] = train['Date'].dt.week

test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day
test['week'] = test['Date'].dt.week

train['StateHoliday'].replace({0 : 4, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)
test['StateHoliday'].replace({0 : 4, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)

In [7]:
# Merging

train = pd.merge(train, shop, how = 'left', on = 'Store')
test = pd.merge(test, shop, how = 'left', on = 'Store')

train['Assortment'].unique()
train['Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)
test['Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)


a = [0, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec']
b = [0, 1, 2, 3]

train['PromoInterval'].replace(dict(zip(a, b)), inplace = True)
test['PromoInterval'].replace(dict(zip(a, b)), inplace = True)

train['StoreType'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)
test['StoreType'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)


train['CompetitionDistance'].fillna(train['CompetitionDistance'].mean(), inplace = True)
test['CompetitionDistance'].fillna(test['CompetitionDistance'].mean(), inplace = True)

train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [8]:
target = 'Sales'
date = 'Date'
customer = 'Customers'

cat_cols = ['Store', 'DayOfWeek', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'year', 'month', 'day', 'dayofweek',
       'week', 'Store_Type', 'Store_Assortment',
       'Store_CompetitionOpenSinceMonth', 'Store_CompetitionOpenSinceYear',
       'Store_Promo2', 'Store_Promo2SinceWeek', 'Store_Promo2SinceYear',
       'Store_PromoInterval']

features = [col for col in train.columns if col not in [target, date, customer]]

In [9]:
trn, val = train_test_split(train, test_size = 0.2, random_state = 1)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test[features]

In [10]:
# Cross Validation for Boosting
def cross_val(regressor, train, test, features, name):
    N_splits = 5
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
        ############ Fitting #############
        _ = regressor.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 50, verbose = 100, eval_metric='rmse')
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = np.sqrt(mean_squared_error((y_val), (val_preds)))
        print(f'\n Root Mean Squared Error for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = np.sqrt(mean_squared_error((target_col), (oofs)))
    print(f'\n\Root Mean Squared Error for oofs is {total_error}')
    
    return oofs, preds

---
# Feature Engineering

In [34]:
def join(train_, test_):
    df = pd.concat([train_, test_], axis = 0).reset_index(drop=True)
    return df

def split(df):
    train_, test_ = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop=True)
    return train_, test_

In [35]:
df = join(train, test)

df['Store_Count'] = df.groupby('Store')['Date'].transform('count')
df['Store_Cus_Count'] = df.groupby('Store')['Customers'].transform('sum')/df['Store_Count']
#df['Store_Mean_Sales'] = df.groupby('Store')['Sales'].transform('mean')
df['Day_of_week_Store_Sale'] = df.groupby(['Store', 'DayOfWeek', 'Promo'])['Sales'].transform('mean')
df['Open_Closed_Sales'] = df.groupby(['Store', 'Open'])['Sales'].transform('mean')
#df["CompetitionOpen"] = ((df["year"] - df["CompetitionOpenSinceYear"]) * 12 + (df["month"] - df["CompetitionOpenSinceMonth"]))

train_feat, test_feat = split(df)

test_feat['Open_Closed_Sales'].fillna(0, inplace=True)

In [13]:
# real_df = join(train_feat, test_feat)

# df = join(train_feat, test_feat)

# def get_elapsed(fld, pre):
#     day1 = np.timedelta64(1, 'D')
#     last_date = np.datetime64()
#     last_store = 0
#     res = []
    
#     for s, v, d in zip(df.Store.values, df[fld].values, df.Date.values): # zip is 300 times faster than using for rows in df.iterrows():
#         if s!= last_store:
#             last_date = np.datetime64()
#             last_store = s
#         if v: last_date = d # keep track of the row if it is a school holiday
#         res.append((d-last_date).astype('timedelta64[D]') / day1) # append the no of days since last school holiday to result(res)
#     df[pre+fld] = res

# columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]
# #df = train[columns]
# df = train[columns].append(test[columns])

# fld = 'SchoolHoliday'
# df = df.sort_values(['Store', 'Date'])
# get_elapsed(fld, 'After')
# df = df.sort_values(['Store', 'Date'], ascending=[True, False]) # sort store in scending and date in descending order.
# get_elapsed(fld, 'Before')

# fld = 'StateHoliday'
# df = df.sort_values(['Store', 'Date'])
# get_elapsed(fld, 'After')
# df = df.sort_values(['Store', 'Date'], ascending=[True, False])
# get_elapsed(fld, 'Before')


# fld = 'Promo'
# df = df.sort_values(['Store', 'Date'])
# get_elapsed(fld, 'After')
# df = df.sort_values(['Store', 'Date'], ascending=[True, False])
# get_elapsed(fld, 'Before')


# df = df.set_index("Date")



# columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

# for o in ['Before', 'After']:
#     for p in columns:
#         a = o+p
#         df[a] = df[a].fillna(0).astype(int)

# bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

# bwd.drop('Store',1,inplace=True)
# bwd.reset_index(inplace=True)

# df = df.merge(bwd, 'left', ["Date", "Store"], suffixes=['', '_bw'])

# df.drop(['Date', 'Store', 'Promo', 'StateHoliday', 'SchoolHoliday'], axis =1, inplace=True)

# df_pro = pd.concat([real_df, df], axis = 1)

# train_feat, test_feat = split(df_pro)

In [36]:
# Changing Datatype

df = join(train_feat, test_feat)

int8_cols = ['DayOfWeek', 'Open', 'Promo','StateHoliday', 'SchoolHoliday', 
'month', 'day','week', 'StoreType', 'Assortment',
'CompetitionOpenSinceMonth','Promo2', 
'Promo2SinceWeek','PromoInterval']

int16_cols = ['year', 'Store', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'Store_Count', 'Store_Cus_Count']

int32_cols = ['CompetitionDistance', 'Day_of_week_Store_Sale', 'Open_Closed_Sales']

def int8(x):
    df[x] = df[x].astype('int8')

def int16(x):
    df[x] = df[x].astype('int16')

def int32(x):
    df[x] = df[x].astype('int32')

for col in int8_cols:
    int8(col)

for col in int16_cols:
    int16(col)

for col in int32_cols:
    int32(col)

train_feat, test_feat = split(df)

---
## Checking

In [37]:
features = [col for col in train_feat.columns if col not in [target, date, customer]]

In [38]:
xgb = XGBRegressor(random_state=1, tree_method='gpu_hist', n_estimators = 500)
lgb = LGBMRegressor(random_state=1, n_estimators = 400)

xgb_params_2 = {'n_estimators': 1496, 'learning_rate': 0.15096576157577118, 'max_depth': 8,
                'colsample_bytree': 0.6155314602051218, 'subsample': 0.7039538177172424, 'min_child_weight': 17}
xgb = XGBRegressor(random_state = 5, tree_method = 'gpu_hist', **xgb_params_2)

In [39]:
# params = {'n_estimators': 1493, 'learning_rate': 0.5708496872557901,
#  'max_depth': 8, 'reg_alpha': 109.98292543369647, 'reg_lambda': 19.460653959854582,
#  'colsample_bytree': 0.995689048773489, 'colsample_bylevel': 0.5636691732005279, 
#  'colsample_bynode': 0.7121333003923356, 'subsample': 0.9856028787866178, 'min_child_weight': 44}

xgb = XGBRegressor(random_state=1, tree_method='gpu_hist', n_jobs = -1, n_estimators = 1400, learning_rate = 0.4,
                  subsample = 0.9, max_depth=9)

In [78]:
trn, val = train_test_split(train_feat, test_size = 0.2, random_state = 1)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test_feat[features]

In [84]:
%%time

xgb_params_2 = {'n_estimators': 1496, 'learning_rate': 0.15096576157577118, 'max_depth': 8, 
                'colsample_bytree': 0.6155314602051218, 'subsample': 0.7039538177172424, 'min_child_weight': 17}

lr = XGBRegressor(random_state=1, n_jobs = -1,tree_method='gpu_hist',**xgb_params_2)
lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val), (preds)))

print(f'mean_squared_log_error is : {error}')

mean_squared_log_error is : 563.8092460725064
Wall time: 1min 23s


In [85]:
xgb_params_2 = {'n_estimators': 1496, 'learning_rate': 0.15096576157577118, 'max_depth': 8, 
                'colsample_bytree': 0.6155314602051218, 'subsample': 0.7039538177172424, 'min_child_weight': 17}

xgb = XGBRegressor(random_state=1, n_jobs = -1,tree_method='gpu_hist',**xgb_params_2)

In [86]:
%%time
xgb_oofs, xgb_preds = cross_val(xgb, train_feat, test_feat, features, 'xgb')
#746
#683.6512919142438
#657.2828283947017 Mean Store Sale --> Oeverfit
#611.5787150171928
#604.9118674736324


================================Fold1===================================
[0]	validation_0-rmse:6015.37451
[100]	validation_0-rmse:676.37201
[200]	validation_0-rmse:635.23108
[300]	validation_0-rmse:618.61304
[400]	validation_0-rmse:607.48248
[500]	validation_0-rmse:600.00806
[600]	validation_0-rmse:595.05273
[700]	validation_0-rmse:590.61493
[800]	validation_0-rmse:587.85748
[900]	validation_0-rmse:585.07501
[1000]	validation_0-rmse:582.08740
[1100]	validation_0-rmse:580.22846
[1200]	validation_0-rmse:578.93652
[1300]	validation_0-rmse:577.74365
[1400]	validation_0-rmse:576.57642
[1495]	validation_0-rmse:575.91174

 Root Mean Squared Error for Validation set is : 575.9095690914025

================================Fold2===================================
[0]	validation_0-rmse:6026.84912
[100]	validation_0-rmse:674.61267
[200]	validation_0-rmse:635.21881
[300]	validation_0-rmse:615.74487
[400]	validation_0-rmse:604.44971
[500]	validation_0-rmse:597.79205
[600]	validation_0-rmse:592.9265

In [87]:
test['Predictions'] = xgb_preds
test.loc[ test['Open'] == 0, 'Predictions'] = 0
preds = test['Predictions']

index = [i for i in range(test.shape[0])]

d = list(zip(index, preds))

ss = pd.DataFrame(d, columns = ['index', 'Sales'])

ss.to_csv(path + "\\xgb_base_tuned.csv", index = False)

model = XGB(est = 500)
first 3 features : 657

3 + Open_Closed_Sales : 643 --> total 4

4 + promo_Day_Store_Sales : 603 --> total 5

5 + Sale/Cus : 531 --> total 6

In [ ]:
xgb_params_2 = {'n_estimators': 1496, 'learning_rate': 0.15096576157577118, 'max_depth': 8, 
                'colsample_bytree': 0.6155314602051218, 'subsample': 0.7039538177172424, 'min_child_weight': 17}

In [48]:
import optuna

def objective(trial, cv=KFold(10, shuffle = True, random_state = 1999)): 

    param_xgb = {
        "random_state": 1,
        'objective': 'reg:squarederror',
        "metric": "rmse",
        # "categorical_feature": cat_indices,
        "verbosity": 0,
        # 'alpha': trial.suggest_int('alpha', 1, 100),
        # 'lambda': trial.suggest_float('lambda', 0.001, 100),
        # 'gamma': trial.suggest_float('gamma', 0.001, 100),
        "n_estimators": trial.suggest_int('n_estimators', 1000, 1600),
        "learning_rate": trial.suggest_float('learning_rate', 0.01, 1),
        # 'num_leaves': trial.suggest_int('num_leaves', 2, 1024),
        'max_depth': trial.suggest_int('max_depth', 5, 9),
         #'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 200),
        #'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 200),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        #'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 100)
        # 'cat_smooth': trial.suggest_float('cat_smooth', 1.0, 50.0) 
        # 'max_delta_step' 
    }

    val_aucs = []
    aucs = []


    X_trn, X_valid, y_trn, y_valid = train_test_split(train_feat[features], train_feat[target], 
                                                    test_size=0.2, random_state=42, shuffle=True)

    model = XGBRegressor(**param_xgb, tree_method='gpu_hist')        

    model.fit(X_trn, y_trn, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100, verbose=200)    

    preds = model.predict(X_valid)

    auc = np.sqrt(mean_squared_error(y_valid, preds)) 

    # auc = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(preds)))
    # aucs.append(auc)

    return auc

In [50]:
#Set to False if you want to skip it
OPTUNA_OPTIMIZATION = True
N_TRIALS = 100

if OPTUNA_OPTIMIZATION:
    study = optuna.create_study(study_name = 'XGB', direction="minimize") 
    
    study.optimize(objective, n_trials=N_TRIALS)

{'n_estimators': 1157, 'learning_rate': 0.1944964205969285, 'max_depth': 9, 
 'colsample_bytree': 0.7318883057123999, 'subsample': 0.9689265877622905, 'min_child_weight': 24}#570

[I 2021-05-06 09:03:05,737] A new study created in memory with name: XGB


[0]	validation_0-rmse:1481.41211
[200]	validation_0-rmse:663.37732
[400]	validation_0-rmse:647.00153
[600]	validation_0-rmse:640.26111
[800]	validation_0-rmse:636.80817
[1000]	validation_0-rmse:635.46057
[1038]	validation_0-rmse:636.21234


[I 2021-05-06 09:03:33,579] Trial 0 finished with value: 635.0005104298594 and parameters: {'n_estimators': 1205, 'learning_rate': 0.8563163332399225, 'max_depth': 6, 'colsample_bytree': 0.9781564975669316, 'subsample': 0.7694900389707997, 'min_child_weight': 75}. Best is trial 0 with value: 635.0005104298594.


[0]	validation_0-rmse:2487.98242
[200]	validation_0-rmse:630.63702
[400]	validation_0-rmse:628.17602
[451]	validation_0-rmse:628.84576


[I 2021-05-06 09:03:56,588] Trial 1 finished with value: 626.9113408091006 and parameters: {'n_estimators': 1289, 'learning_rate': 0.6730389416402681, 'max_depth': 8, 'colsample_bytree': 0.6820533545417058, 'subsample': 0.9074827715096094, 'min_child_weight': 7}. Best is trial 1 with value: 626.9113408091006.


...

[I 2021-05-06 10:50:50,440] Trial 87 finished with value: 560.0452863982138 and parameters: {'n_estimators': 1516, 'learning_rate': 0.12560968165665126, 'max_depth': 9, 'colsample_bytree': 0.8567750097749876, 'subsample': 0.9986568366234397, 'min_child_weight': 7}. Best is trial 79 with value: 558.6676404337702.


[0]	validation_0-rmse:6964.46289
[200]	validation_0-rmse:1059.62647
[400]	validation_0-rmse:756.76013
[600]	validation_0-rmse:705.29944
[800]	validation_0-rmse:674.93066
[1000]	validation_0-rmse:657.71735
[1200]	validation_0-rmse:645.10284
[1400]	validation_0-rmse:634.92596
[1515]	validation_0-rmse:629.95612


[I 2021-05-06 10:52:51,973] Trial 88 finished with value: 629.9561063263142 and parameters: {'n_estimators': 1516, 'learning_rate': 0.011836038406695629, 'max_depth': 9, 'colsample_bytree': 0.8526521401970661, 'subsample': 0.9636626854559199, 'min_child_weight': 7}. Best is trial 79 with value: 558.6676404337702.


[0]	validation_0-rmse:6212.62842
[200]	validation_0-rmse:618.63373
[400]	validation_0-rmse:587.74341


KeyboardInterrupt: 

---
# LGBM

In [46]:
import optuna

def objective(trial, cv=KFold(10, shuffle = True, random_state = 1999)): 

    param_lgb = {
        "random_state": 1,
        #'objective': 'reg:squarederror',
        #"metric": "rmse",
        # "categorical_feature": cat_indices,
        #"verbosity": 0,
        # 'alpha': trial.suggest_int('alpha', 1, 100),
        # 'lambda': trial.suggest_float('lambda', 0.001, 100),
        # 'gamma': trial.suggest_float('gamma', 0.001, 100),
        "n_estimators": trial.suggest_int('n_estimators', 1000, 3000),
        "learning_rate": trial.suggest_float('learning_rate', 0.01, 1),
        'num_leaves': trial.suggest_int('num_leaves', 30, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 9),
         #'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 200),
        #'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 200),
        #'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        #'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        #'min_child_weight': trial.suggest_int('min_child_weight', 1, 100)
        # 'cat_smooth': trial.suggest_float('cat_smooth', 1.0, 50.0) 
        # 'max_delta_step' 
    }

    val_aucs = []
    aucs = []


    X_trn, X_valid, y_trn, y_valid = train_test_split(train_feat[features], train_feat[target], 
                                                    test_size=0.2, random_state=42, shuffle=True)

    model = LGBMRegressor(**param_lgb)        

    model.fit(X_trn, y_trn, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100, verbose=200, eval_metric = 'rmse')    

    preds = model.predict(X_valid)

    auc = np.sqrt(mean_squared_error(y_valid, preds)) 

    # auc = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(preds)))
    # aucs.append(auc)

    return auc

In [47]:
#Set to False if you want to skip it
OPTUNA_OPTIMIZATION = True
N_TRIALS = 100 

if OPTUNA_OPTIMIZATION:
    study = optuna.create_study(study_name = 'LGB', direction="minimize") 
    
    study.optimize(objective, n_trials=N_TRIALS)

[I 2021-05-06 06:47:40,382] A new study created in memory with name: LGB


Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 623.609	valid_0's l2: 388888
[400]	valid_0's rmse: 608.865	valid_0's l2: 370717
[600]	valid_0's rmse: 606.46	valid_0's l2: 367794
Early stopping, best iteration is:
[665]	valid_0's rmse: 606.228	valid_0's l2: 367512


[I 2021-05-06 06:48:01,789] Trial 0 finished with value: 606.2277674095544 and parameters: {'n_estimators': 1497, 'learning_rate': 0.680779582311707, 'num_leaves': 445, 'max_depth': 7, 'subsample': 0.8078848640956956}. Best is trial 0 with value: 606.2277674095544.


Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 655.748	valid_0's l2: 430005
[400]	valid_0's rmse: 619.076	valid_0's l2: 383256
[600]	valid_0's rmse: 600.626	valid_0's l2: 360751
[800]	valid_0's rmse: 591.515	valid_0's l2: 349890
[1000]	valid_0's rmse: 584.689	valid_0's l2: 341861
[1200]	valid_0's rmse: 578.796	valid_0's l2: 335005
[1400]	valid_0's rmse: 574.319	valid_0's l2: 329842
Did not meet early stopping. Best iteration is:
[1594]	valid_0's rmse: 571.521	valid_0's l2: 326637


[I 2021-05-06 06:48:55,176] Trial 1 finished with value: 571.5212387779153 and parameters: {'n_estimators': 1594, 'learning_rate': 0.08054501751892403, 'num_leaves': 414, 'max_depth': 8, 'subsample': 0.639722771474805}. Best is trial 1 with value: 571.5212387779153.


...

[I 2021-05-06 08:19:30,535] Trial 97 finished with value: 563.6086991338769 and parameters: {'n_estimators': 2528, 'learning_rate': 0.18520735126557947, 'num_leaves': 113, 'max_depth': 9, 'subsample': 0.9868331371756641}. Best is trial 74 with value: 559.6492544947287.


Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 734.858	valid_0's l2: 540016
[400]	valid_0's rmse: 697.42	valid_0's l2: 486394
[600]	valid_0's rmse: 679.805	valid_0's l2: 462135
[800]	valid_0's rmse: 666.642	valid_0's l2: 444412
[1000]	valid_0's rmse: 656.344	valid_0's l2: 430787
[1200]	valid_0's rmse: 648.364	valid_0's l2: 420376
[1400]	valid_0's rmse: 641.775	valid_0's l2: 411875
[1600]	valid_0's rmse: 635.596	valid_0's l2: 403983
[1800]	valid_0's rmse: 630.514	valid_0's l2: 397548
[2000]	valid_0's rmse: 626.595	valid_0's l2: 392621
[2200]	valid_0's rmse: 622.648	valid_0's l2: 387690
[2400]	valid_0's rmse: 619.38	valid_0's l2: 383631
Did not meet early stopping. Best iteration is:
[2575]	valid_0's rmse: 616.362	valid_0's l2: 379902


[I 2021-05-06 08:20:14,460] Trial 98 finished with value: 616.3617403142224 and parameters: {'n_estimators': 2575, 'learning_rate': 0.096952129942236, 'num_leaves': 94, 'max_depth': 5, 'subsample': 0.9021073649577028}. Best is trial 74 with value: 559.6492544947287.


Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 676.368	valid_0's l2: 457473
[400]	valid_0's rmse: 633.989	valid_0's l2: 401942
[600]	valid_0's rmse: 616.09	valid_0's l2: 379567
[800]	valid_0's rmse: 605.217	valid_0's l2: 366288
[1000]	valid_0's rmse: 596.451	valid_0's l2: 355754
[1200]	valid_0's rmse: 589.33	valid_0's l2: 347310
[1400]	valid_0's rmse: 583.722	valid_0's l2: 340731
[1600]	valid_0's rmse: 579.31	valid_0's l2: 335600
[1800]	valid_0's rmse: 575.643	valid_0's l2: 331365
[2000]	valid_0's rmse: 572.866	valid_0's l2: 328175
[2200]	valid_0's rmse: 570.56	valid_0's l2: 325539
[2400]	valid_0's rmse: 568.523	valid_0's l2: 323218
Did not meet early stopping. Best iteration is:
[2490]	valid_0's rmse: 567.83	valid_0's l2: 322431


[I 2021-05-06 08:21:31,747] Trial 99 finished with value: 567.8298524488127 and parameters: {'n_estimators': 2493, 'learning_rate': 0.04707518039214978, 'num_leaves': 205, 'max_depth': 9, 'subsample': 0.9375093312609258}. Best is trial 74 with value: 559.6492544947287.


---
# Catboost

In [44]:
import optuna

def objective(trial, cv=KFold(10, shuffle = True, random_state = 1999)): 

    param_cat = {
        "random_state": 1,
        #'objective': 'reg:squarederror',
        #"metric": "rmse",
        # "categorical_feature": cat_indices,
        #"verbosity": 0,
        # 'alpha': trial.suggest_int('alpha', 1, 100),
        # 'lambda': trial.suggest_float('lambda', 0.001, 100),
        # 'gamma': trial.suggest_float('gamma', 0.001, 100),
        "n_estimators": trial.suggest_int('n_estimators', 1000, 3000),
        "learning_rate": trial.suggest_float('learning_rate', 0.01, 1),
        #'num_leaves': trial.suggest_int('num_leaves', 30, 500),
        'depth': trial.suggest_int('depth', 3, 10),
         #'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 200),
        #'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 200),
        #'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        #'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'rsm': trial.suggest_float('rsm', 0.5, 1.0),
        #'min_child_weight': trial.suggest_int('min_child_weight', 1, 100)
        # 'cat_smooth': trial.suggest_float('cat_smooth', 1.0, 50.0) 
        # 'max_delta_step' 
    }

    val_aucs = []
    aucs = []


    X_trn, X_valid, y_trn, y_valid = train_test_split(train_feat[features], train_feat[target], 
                                                    test_size=0.2, random_state=42, shuffle=True)

    model = CatBoostRegressor(**param_cat)        

    model.fit(X_trn, y_trn, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100, verbose=200)    

    preds = model.predict(X_valid)

    auc = np.sqrt(mean_squared_error(y_valid, preds)) 

    # auc = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(preds)))
    # aucs.append(auc)

    return auc

In [45]:
#Set to False if you want to skip it
OPTUNA_OPTIMIZATION = True
N_TRIALS = 100 

if OPTUNA_OPTIMIZATION:
    study = optuna.create_study(study_name = 'CAT', direction="minimize") 
    
    study.optimize(objective, n_trials=N_TRIALS)

[I 2021-05-06 02:21:58,709] A new study created in memory with name: CAT


0:	learn: 1675.4270942	test: 1675.6879941	best: 1675.6879941 (0)	total: 210ms	remaining: 8m 12s
200:	learn: 664.4406813	test: 684.6913852	best: 684.6913852 (200)	total: 12.1s	remaining: 2m 9s
400:	learn: 620.3271492	test: 651.8298782	best: 651.8298782 (400)	total: 25.6s	remaining: 2m 4s
600:	learn: 594.4394118	test: 635.5470526	best: 635.5470526 (600)	total: 37.9s	remaining: 1m 50s
800:	learn: 576.1887212	test: 625.1333112	best: 625.1333112 (800)	total: 49.1s	remaining: 1m 34s
1000:	learn: 561.1063462	test: 618.5934956	best: 618.2938311 (999)	total: 1m	remaining: 1m 21s
1200:	learn: 549.4787205	test: 613.6783326	best: 613.6783326 (1200)	total: 1m 11s	remaining: 1m 8s
1400:	learn: 539.5849070	test: 610.5257128	best: 610.4989088 (1399)	total: 1m 23s	remaining: 56.1s
1600:	learn: 530.6596451	test: 606.2748810	best: 606.2748221 (1599)	total: 1m 34s	remaining: 44.1s
1800:	learn: 522.6463142	test: 603.5008612	best: 603.5008612 (1800)	total: 1m 45s	remaining: 32.1s
2000:	learn: 515.3103564	te

[I 2021-05-06 02:24:17,222] Trial 0 finished with value: 598.9700955684492 and parameters: {'n_estimators': 2347, 'learning_rate': 0.7915402335706753, 'depth': 6, 'rsm': 0.5193243574504376}. Best is trial 0 with value: 598.9700955684492.


2346:	learn: 504.2792607	test: 598.9700956	best: 598.9700956 (2346)	total: 2m 17s	remaining: 0us

bestTest = 598.9700956
bestIteration = 2346

0:	learn: 2478.9903267	test: 2477.6672361	best: 2477.6672361 (0)	total: 50.5ms	remaining: 1m 19s
200:	learn: 677.9608228	test: 692.0586093	best: 692.0586093 (200)	total: 11.8s	remaining: 1m 20s
400:	learn: 633.3116207	test: 656.2402168	best: 656.2402168 (400)	total: 23.7s	remaining: 1m 8s
600:	learn: 608.5087759	test: 638.1466579	best: 638.1466579 (600)	total: 35.7s	remaining: 57.5s
800:	learn: 590.2115090	test: 626.3934110	best: 626.3934110 (800)	total: 47.7s	remaining: 45.8s
1000:	learn: 575.8823082	test: 617.3056098	best: 617.3056098 (1000)	total: 59.6s	remaining: 33.8s
1200:	learn: 564.1189696	test: 610.9527072	best: 610.9337714 (1198)	total: 1m 11s	remaining: 21.8s
1400:	learn: 554.5284267	test: 606.1995555	best: 606.1995555 (1400)	total: 1m 23s	remaining: 9.97s


[I 2021-05-06 02:25:51,064] Trial 1 finished with value: 602.5077920139041 and parameters: {'n_estimators': 1569, 'learning_rate': 0.4434581058198212, 'depth': 6, 'rsm': 0.755530062121413}. Best is trial 0 with value: 598.9700955684492.


...

[I 2021-05-06 06:41:10,801] Trial 97 finished with value: 564.0379182123477 and parameters: {'n_estimators': 2995, 'learning_rate': 0.3158225354422536, 'depth': 8, 'rsm': 0.9424393569237287}. Best is trial 92 with value: 557.4821953237079.


0:	learn: 2972.4584610	test: 2972.0817099	best: 2972.0817099 (0)	total: 66.8ms	remaining: 2m 52s
200:	learn: 626.5172918	test: 652.2209376	best: 652.2209376 (200)	total: 15.8s	remaining: 3m 6s
400:	learn: 575.4612863	test: 617.2921884	best: 617.2921884 (400)	total: 31.6s	remaining: 2m 51s
600:	learn: 542.6432442	test: 598.7729320	best: 598.7729320 (600)	total: 47.4s	remaining: 2m 35s
800:	learn: 519.3279949	test: 588.1102249	best: 588.1102249 (800)	total: 1m 2s	remaining: 2m 19s
1000:	learn: 499.5952317	test: 580.6955613	best: 580.6955613 (1000)	total: 1m 18s	remaining: 2m 3s
1200:	learn: 482.8380915	test: 574.5858331	best: 574.5858331 (1200)	total: 1m 34s	remaining: 1m 47s
1400:	learn: 468.9077256	test: 571.0175938	best: 571.0175938 (1400)	total: 1m 49s	remaining: 1m 32s
1600:	learn: 456.3779531	test: 567.4208847	best: 567.4208847 (1600)	total: 2m 5s	remaining: 1m 16s
1800:	learn: 445.2100741	test: 565.1734331	best: 565.1734331 (1800)	total: 2m 21s	remaining: 1m
2000:	learn: 435.56726

[I 2021-05-06 06:44:34,547] Trial 98 finished with value: 559.4148324656448 and parameters: {'n_estimators': 2576, 'learning_rate': 0.2763094289655646, 'depth': 9, 'rsm': 0.9358389412411411}. Best is trial 92 with value: 557.4821953237079.


0:	learn: 2753.4045616	test: 2751.8514001	best: 2751.8514001 (0)	total: 88.1ms	remaining: 4m 19s
200:	learn: 612.7814593	test: 642.3400761	best: 642.3400761 (200)	total: 15.3s	remaining: 3m 28s
400:	learn: 559.4765695	test: 607.9779240	best: 607.9779240 (400)	total: 30.5s	remaining: 3m 13s
600:	learn: 525.4230737	test: 592.5305147	best: 592.5305147 (600)	total: 45.5s	remaining: 2m 58s
800:	learn: 499.8136231	test: 583.2426825	best: 583.2426825 (800)	total: 1m	remaining: 2m 43s
1000:	learn: 481.0578798	test: 577.4951573	best: 577.4951573 (1000)	total: 1m 16s	remaining: 2m 28s
1200:	learn: 464.4940257	test: 572.5165124	best: 572.5165124 (1200)	total: 1m 31s	remaining: 2m 12s
1400:	learn: 450.4204439	test: 569.7188350	best: 569.7188350 (1400)	total: 1m 46s	remaining: 1m 57s
1600:	learn: 438.2891277	test: 567.8671782	best: 567.8671782 (1600)	total: 2m 1s	remaining: 1m 42s
1800:	learn: 427.0727479	test: 566.1048568	best: 566.0993026 (1799)	total: 2m 17s	remaining: 1m 27s
2000:	learn: 417.12

[I 2021-05-06 06:47:40,355] Trial 99 finished with value: 563.4085554206642 and parameters: {'n_estimators': 2951, 'learning_rate': 0.342249656894165, 'depth': 9, 'rsm': 0.9123736457258511}. Best is trial 92 with value: 557.4821953237079.


params = {'n_estimators': 1573, 'learning_rate': 0.47014065599594035,
 'max_depth': 8, 'reg_alpha': 198.326716938535, 'reg_lambda': 125.91594195807608, 
 'colsample_bytree': 0.9543001032773433, 'colsample_bylevel': 0.675896785667841, 
 'colsample_bynode': 0.8506402529997479, 'subsample': 0.9973968191944973, 'min_child_weight': 98}

xgb = XGBRegressor(random_state=1, tree_method = 'gpu_hist', **params)

In [155]:
test['Predictions'] = xgb_preds
test.loc[ test['Open'] == 0, 'Predictions'] = 0
preds = test['Predictions']

index = [i for i in range(test.shape[0])]

d = list(zip(index, preds))

ss = pd.DataFrame(d, columns = ['index', 'Sales'])

ss.to_csv(path + "\\new_feats.csv", index = False)

In [161]:

ss.to_csv(path + "\\new_feats.csv", index = False)